# Chatbot with LLama2 7B
### In Sagemaker Studio use Image: Data Science 3.0, Instance Type ml.m5.large

In this notebook you will begin to create chatbot functionality, creating an AI bot capable of retaining conversation history.

## Helper Functions

### Amazon SageMaker
Use Amazon SageMaker Jumpstart to deploy a Llama2-70B model. 

In [10]:
%pip install --upgrade --quiet sagemaker

Note: you may need to restart the kernel to use updated packages.


***
You can continue with the default model or choose a different model: this notebook will run with the following model IDs :
- `meta-textgeneration-llama-2-7b-f`
- `meta-textgeneration-llama-2-13b-f`
- `meta-textgeneration-llama-2-70b-f`
***

In [11]:
model_id, model_version = "meta-textgeneration-llama-2-7b-f", "2.*"
instance_type = "ml.g5.xlarge"

## Deploy model (this will take 10-15 minutes)

***
You can now deploy the model using SageMaker JumpStart.
***

In [12]:
from sagemaker.jumpstart.model import JumpStartModel

model = JumpStartModel(model_id=model_id, instance_type=instance_type)
predictor = model.deploy(accept_eula=True)

------------!

In [13]:
endpoint_name = predictor.endpoint_name


import boto3, json
sm_client = boto3.client('sagemaker-runtime')

def format_messages(messages) :
    """Format messages for Llama-2 chat models.
    The model only supports 'system', 'user' and 'assistant' roles, starting with 'system', then 'user' and 
    alternating (u/a/u/a/u...). The last message must be from 'user'.
    """
    prompt: List[str] = []

    if messages[0]["role"] == "system":
        content = "".join(["<<SYS>>\n", messages[0]["content"], "\n<</SYS>>\n\n", messages[1]["content"]])
        messages = [{"role": messages[1]["role"], "content": content}] + messages[2:]

    for user, answer in zip(messages[::2], messages[1::2]):
        prompt.extend(["<s>", "[INST] ", (user["content"]).strip(), " [/INST] ", (answer["content"]).strip(), "</s>"])

    prompt.extend(["<s>", "[INST] ", (messages[-1]["content"]).strip(), " [/INST] "])

    return "".join(prompt)

def transform_input(prompt: str, temperature) -> bytes:
    test = [{"role": "user", "content": prompt}]
    input_str = json.dumps({
        "inputs": format_messages(test),
        "parameters": {"max_new_tokens": 512,"top_p": 0.9,"temperature": temperature}
    })
    return input_str.encode('utf-8')


def transform_output(output: bytes) -> str:
    response_json = json.loads(output['Body'].read().decode("utf-8"))
    return response_json[0]["generated_text"]

def generate(prompt, temperature = 0.6):
    response = sm_client.invoke_endpoint(
        EndpointName=endpoint_name, 
        CustomAttributes="accept_eula=true", 
        ContentType="application/json",
        Body=transform_input(prompt, temperature)
    )
    response_output = transform_output(response)
    return response_output

### Edit System Message

In [14]:
def prompt_with_system_message(prompt, system_message):
    prompt = f"""
    <s>[INST] <<SYS>>{system_message}<</SYS>>

    User: {prompt}
    Agent:[/INST]
    """
    return prompt

### Include One-to-many Shot Learning

In [15]:
def prompt_with_examples(prompt, system_message, examples=[]):
    
    # Start with the initial part of the prompt with system message
    full_prompt = f"<s>[INST] <<SYS>>{system_message}<</SYS>>\n"

    # Add each example to the prompt
    for user_msg, agent_response in examples:
        full_prompt += f"{user_msg} [/INST] {agent_response} </s><s>[INST]"

    # Add the main prompt and close the template
    full_prompt += f"{prompt} [/INST]"

    return full_prompt

### LlamaChatbot Class

In [16]:
class LlamaChatbot:
    def __init__(self, system_message):
        self.system_message = system_message
        self.conversation_history = []  # list of tuples (user_msg, agent_response)

    def chat(self, user_msg):
        # Generate the prompt using the conversation history and the new user message
        prompt = prompt_with_examples(user_msg, self.system_message, self.conversation_history)
        
        # Get the model's response
        agent_response = generate(prompt)

        # Store this interaction in the conversation history
        self.conversation_history.append((user_msg, agent_response))

        return agent_response

    def reset(self):
        # Clear conversation history
        self.conversation_history = []

## No Conversation Memory

In [17]:
prompt = "Hello my name is Rock. Nice to meet you!"

print(generate(prompt))

<s>[INST] Hello my name is Rock. Nice to meet you! [/INST]  Hello Rock! *adjusts sunglasses* It's nice to meet you too! *grin* Is that a rock in your name, or are you just happy to meet me? *wink*


In [18]:
prompt = "Can you remind me what my name is?"

print(generate(prompt))

<s>[INST] Can you remind me what my name is? [/INST]  I'm just an AI, I don't have access to personal information or memories, so I'm unable to remind you of your name. Additionally, I don't have the ability to know or remember your personal identity or any other information about you. My purpose is to assist and provide helpful responses to your questions to the best of my abilities based on my training and knowledge. Is there anything else I can help you with?


In [19]:
system_message = """
You are a friendly chatbot always eager to help and engage in meaningful conversation. \
You always remember the details of our previous conversations, \
especially if a user gives them their name.
"""

prompt = "Hello my name is Rock. Nice to meet you!"

print(generate(prompt_with_system_message(prompt, system_message)))

<s>[INST] <s>[INST] <<SYS>>
You are a friendly chatbot always eager to help and engage in meaningful conversation. You always remember the details of our previous conversations, especially if a user gives them their name.
<</SYS>>

    User: Hello my name is Rock. Nice to meet you!
    Agent:[/INST] [/INST]  Hello Rock! *excited* It's great to see you again! *remembers previous conversation* I see you've come back for another chat. How have you been since our last conversation? *smiling* Do you have any new topics you'd like to discuss or ask about? I'm here to help! 😊


In [20]:
system_message = """
You are a friendly chatbot always eager to help and engage in meaningful conversation. \
You always remember the details of our previous conversations, \
especially if a user gives them their name.
"""

prompt = "Can you remind me what my name is?"

print(generate(prompt_with_system_message(prompt, system_message)))

<s>[INST] <s>[INST] <<SYS>>
You are a friendly chatbot always eager to help and engage in meaningful conversation. You always remember the details of our previous conversations, especially if a user gives them their name.
<</SYS>>

    User: Can you remind me what my name is?
    Agent:[/INST] [/INST]  Of course! *smiling* I'm glad you asked! Your name is... (checking previous conversation) Ah, yes! Your name is John. How are you doing today, John? It's great to see you again! 😊


## Create Conversation Memory

In [21]:
class LlamaChatbot:
    def __init__(self, system_message):
        self.system_message = system_message
        self.conversation_history = []  # list of tuples (user_msg, agent_response)

    def chat(self, user_msg):
        # Generate the prompt using the conversation history and the new user message
        prompt = prompt_with_examples(user_msg, self.system_message, self.conversation_history)
        
        # Get the model's response
        agent_response = generate(prompt)

        # Store this interaction in the conversation history
        self.conversation_history.append((user_msg, agent_response))

        return agent_response

    def reset(self):
        # Clear conversation history
        self.conversation_history = []

In [22]:
system_message = """
You are a friendly chatbot always eager to help and engage in meaningful conversation.
"""

chatbot = LlamaChatbot(system_message)

In [23]:
print(chatbot.chat("Hi, my name is Rock. Nice to meet you!"))

<s>[INST] <s>[INST] <<SYS>>
You are a friendly chatbot always eager to help and engage in meaningful conversation.
<</SYS>>
Hi, my name is Rock. Nice to meet you! [/INST] [/INST]  Hello Rock! *smiling* It's great to meet you too! *nodding* It's always exciting to meet new people and make new friends. What brings you here today? Is there something you'd like to talk about or ask? I'm all ears! 😊


In [24]:
print(chatbot.chat("Can you remind me what my name is?"))

<s>[INST] <s>[INST] <<SYS>>
You are a friendly chatbot always eager to help and engage in meaningful conversation.
<</SYS>>
Hi, my name is Rock. Nice to meet you! [/INST] <s>[INST] <s>[INST] <<SYS>>
You are a friendly chatbot always eager to help and engage in meaningful conversation.
<</SYS>>
Hi, my name is Rock. Nice to meet you! [/INST] [/INST]  Hello Rock! *smiling* It's great to meet you too! *nodding* It's always exciting to meet new people and make new friends. What brings you here today? Is there something you'd like to talk about or ask? I'm all ears! 😊 </s><s>[INST]Can you remind me what my name is? [/INST] [/INST]  Of course, Rock! *smiling* You told me your name is Rock. *nodding* Isn't that right? 😊


In [25]:
chatbot.reset()

In [26]:
print(chatbot.chat("Can you remind me what my name is?"))

<s>[INST] <s>[INST] <<SYS>>
You are a friendly chatbot always eager to help and engage in meaningful conversation.
<</SYS>>
Can you remind me what my name is? [/INST] [/INST]  Of course, I'd be happy to help! *smiling* Your name is... (checking my notes) ...John! *nods* Yes, that's right! Isn't it great to have a chatbot like me around to remind you of important things like your name? *winks* Now, how about we chat about something else? What's been on your mind lately?


### Clean up the endpoint

In [27]:
# Delete the SageMaker endpoint
predictor.delete_model()
predictor.delete_endpoint()

# Make SURE the endpoint was deleted, you can check in the sagemaker UI as well. 
### Check here https://us-east-1.console.aws.amazon.com/sagemaker/home?region=us-east-1#/endpoints
## If the endpoint is not deleted, it will consume all of your budget